In [ ]:
import pandas as pd
import numpy as np

import re
import nltk
from string import punctuation

nltk.download('stopwords')
punctuation = list(punctuation)

# these characters are reminants of typos
punctuation.extend(["''", '``'])

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk import FreqDist

pd.options.display.max_columns = 50
pd.options.display.max_rows = 50

In [ ]:
# load in files
contracts = pd.read_csv('../data/contracts.csv', low_memory=False)

categories_csv = pd.read_csv('../data/categories/gov-objs-2022.csv', dtype = str, low_memory=False)

In [ ]:
# drop excess rows from `contacts.csv`
contracts = contracts.drop(['vendor_postal_code', 'description_fr', 'comments_fr', 'additional_comments_fr', 'vendor_postal_code'], axis=1)

In [ ]:
# drop excess rows from `gov-objs-2022.csv`
categories_csv = categories_csv.drop(['Group-Groupe_code_OBJ-ART', 'Group_name-Groupe_nom_OBJ-ART', 'Category_sub-division-Sous-divisiondecatégorie_code_OBJ-ART', 'Category_sub-division_name-Sous-divisiondecatégorie_nom_OBJ-ART', 'Notes', 'Department_specific-Particulier_au_ministère'], axis=1)

In [ ]:
# splitting `categories` into individual dfs with same column names so they can be concatenated
df1 = categories_csv[['Category-Catégorie_code_OBJ-ART', 'Category_name-Catégorie_nom_OBJ-ART']].copy()
df1.rename(columns={'Category-Catégorie_code_OBJ-ART': 'eco_obj_code',
                    'Category_name-Catégorie_nom_OBJ-ART': 'name'}, inplace=True)

df2 = categories_csv[['Sub-category-Sous-catégorie_code_OBJ-ART', 'Sub-category_name-Sous-catégorie_nom_OBJ-ART', 'Sub-category-Sous-catégorie_description_OBJ-ART']].copy()
df2.rename(columns={'Sub-category-Sous-catégorie_code_OBJ-ART': 'eco_obj_code',
                    'Sub-category_name-Sous-catégorie_nom_OBJ-ART': 'name',
                    'Sub-category-Sous-catégorie_description_OBJ-ART': 'description'}, inplace=True)

df3 = categories_csv[['Reportingobject-Articlederapport_code_OBJ-ART', 'Reportingobject_name-Articlederapport_nom_OBJ-ART']].copy()
df3.rename(columns={'Reportingobject-Articlederapport_code_OBJ-ART': 'eco_obj_code',
                    'Reportingobject_name-Articlederapport_nom_OBJ-ART': 'name'}, inplace=True)

df4 = categories_csv[['Code_OBJ-ART', 'Name-Nom_OBJ-ART', 'Description_OBJ-ART']].copy()
df4.rename(columns={'Code_OBJ-ART': 'eco_obj_code',
                    'Name-Nom_OBJ-ART': 'name',
                    'Description_OBJ-ART': 'description'}, inplace=True)

categories = pd.concat([df1, df2, df3, df4])

In [ ]:
# getting only unique EOCs for categorisation
categories = categories.drop_duplicates(subset=['eco_obj_code'], ignore_index=True)
categories

## Creating table of EOCs which exist in `contracts.csv` for manual categorization

In [ ]:
# TEMP COPY OF CONTRACTS DF FOR EXPERIMENTING BELOW
temp_contracts = contracts
temp_cats = categories

In [ ]:
def clean_desc(desc):
    if type(desc) is str:
        tokens = word_tokenize(desc.lower())

        stwords = stopwords.words('english')
        newDesc = [token for token in tokens if token not in stwords and token not in punctuation]
        
        return newDesc

    

# temp_cats['name'] = temp_cats['name'].apply(clean_desc)


In [ ]:
for row_index, eco_code in enumerate(temp_contracts['economic_object_code']):
    # check for rows with no EOC or incorrectly formatted EOCs
    if pd.isna(eco_code) or eco_code[0].isdigit() == False:
        if pd.isna(temp_contracts.loc[row_index]['description_en']) == False:
            split_desc = clean_desc(temp_contracts.loc[row_index]['description_en'])
            for str_index, s in enumerate(split_desc):
                if s.isdigit():
                    if s[0] == '0':
                        # easy match: if number begins with 0 then it can be presumed to be an object code
                        temp_contracts.loc[row_index, 'economic_object_code'] = s
                    elif len(s) >= 3 and str_index == 0:
                        # harder match: what about the 4 digit codes or the ones that don't follow the correct format?
                        # solution: if number is >= 3 digits in length and the number is at index 0, then match number to categories 
                        # s = normalise_codes(s)
                        temp_contracts.loc[row_index, 'economic_object_code'] = s
                    elif str_index == 0:
                        # case for if digit is in first position (likely to be EOC)
                        temp_contracts.loc[row_index, 'economic_object_code'] = s


In [ ]:
temp_cats['name'] = temp_cats['name'].apply(clean_desc)

# final check to match descriptions of rows with no EOC or number in description
for row_index, eco_code in enumerate(temp_contracts['economic_object_code']):
        split_desc = clean_desc(temp_contracts.loc[row_index]['description_en'])
        if pd.isna(eco_code) or eco_code[0].isdigit() == False:
            matches = []
            for i, d in enumerate(temp_cats['name']):
                if split_desc == d:
                    matches.append(temp_cats.loc[i])
            if len(matches) > 0:
                if matches[0]['name'] != None:
                    temp_contracts.loc[row_index, 'economic_object_code'] = matches[0]['eco_obj_code']



In [ ]:
forCate = temp_contracts.drop_duplicates(subset=['economic_object_code'], ignore_index=True)

In [ ]:
forSean = pd.read_csv('../data/categories/REVISED_economic_object_codes_to_category.csv', dtype = str, low_memory=False)

forSean['example_description_en'] = forSean['example_description_en'].apply(clean_desc)

forCate['category'] = pd.Series()

for row_index, eco_code in enumerate(forCate['economic_object_code']):
    split_desc = clean_desc(forCate.loc[row_index]['description_en'])
    matches = []
    for i, d in enumerate(forSean['example_description_en']):
        if split_desc == d:
            matches.append(forSean.loc[i])
    if len(matches) > 0:
        if matches[0]['example_description_en'] != None:
            print(matches[0])
            forCate.loc[row_index, 'category'] = matches[0]['category']


In [ ]:
forCate = forCate[['economic_object_code', 'description_en', 'category']].copy()
forCate
# forCate.to_csv('for-Sean-revision.csv')

## Code for Categorisation

In [ ]:
cat_contracts = temp_contracts
cat_contracts.head(5)

In [ ]:
# EOCs exported incorrectly from after Sean manual categorization
# can be amended by row index
classifications1 = pd.read_csv('manual_categorisation.csv', dtype = str, low_memory=False)

for row_index, eco_code in enumerate(classifications1['economic_object_code']):
    classifications1.loc[row_index, 'economic_object_code'] = forCate.loc[row_index, 'economic_object_code']

In [ ]:
# fix Excel removal of leading 0s from Sean categorization pt2
classifications2 = pd.read_csv('../data/categories/economic_object_codes_to_category.csv', dtype = str, low_memory=False)

for row_index, eco_code in enumerate(classifications2['economic_object_code']):
    if eco_code not in list(classifications1['economic_object_code']):
        mod_eco_code = '0' + eco_code
        if mod_eco_code in list(classifications1['economic_object_code']):
            classifications2.loc[row_index, 'economic_object_code'] = mod_eco_code

In [ ]:
# fix discrepancies
des_r = pd.read_csv('descrepancies_reconciled.csv', dtype = str, low_memory=False)

for row_index, eco_code in enumerate(des_r['economic_object_code']):
    if eco_code not in list(classifications1['economic_object_code']):
        mod_eco_code = '0' + eco_code
        if mod_eco_code in list(classifications1['economic_object_code']):
            des_r.loc[row_index, 'economic_object_code'] = mod_eco_code

merged_sb_class = pd.merge(classifications2, des_r, how="outer", on=["economic_object_code"])

for row_index, cate in enumerate(merged_sb_class['category_latest']):
    if type(cate) == str:
        merged_sb_class.loc[row_index, "category"] = cate

classifications2 = merged_sb_class[['economic_object_code', 'category']].copy()

merged = pd.merge(classifications1, classifications2, how="outer", on=["economic_object_code"])

for row_index, cate in enumerate(merged['category_x']):
    if type(cate) != str:
        if type(merged.loc[row_index, "category_y"]) == str:
           merged.loc[row_index, "category_x"] = merged.loc[row_index, "category_y"]
        
classifications = merged.drop_duplicates(subset=['economic_object_code'])
classifications = classifications[['economic_object_code', 'description_en', 'category_x']].copy()
classifications = classifications.rename(columns={'category_x':'category'})

# merged.loc[merged['category_x'].notna()]
# merged = merged.loc[merged['category_cb'] != merged['category_sb']]
# merged = merged.loc[merged['category_cb'].notna() & merged['category_sb'].notna()]

# merged.to_csv('descrepancies.csv')

In [ ]:
classifications

In [ ]:
# create column for classification
cat_contracts.insert(6,'category', '')

In [ ]:
def classify(code):
    find_class = classifications.loc[classifications['economic_object_code'] == code]
    return find_class

for row_index, eco_code in enumerate(cat_contracts['economic_object_code']):
    if pd.isna(eco_code) == False:
        loc_class = classify(eco_code)
        if loc_class.empty:
            mod_code = ''
            if eco_code[0] == '0':
                mod_code = eco_code[1:]
            else:
                mod_code = '0' + eco_code

            loc_class = classify(mod_code)

            if loc_class.empty == False:
                cat_contracts.loc[row_index, 'category'] = loc_class.iloc[0]['category']
            else:
                cat_contracts.loc[row_index, 'category'] = np.NaN

        else:
            cat_contracts.loc[row_index, 'category'] = loc_class.iloc[0]['category']
        
        print(eco_code)


In [ ]:
# we will now export the contracts data as is and attempt to create a text classifier for the remaining 204416 rows without any EOC or description match 
cat_contracts.to_csv('rule_based_contracts_v1.csv')

# PREVIOUS RULE-BASED CLASSIFIER

In [ ]:
# find word frequency for each category to classify rows with no EOC later
unique_classifiers = [c for c in cat_contracts['category'].unique() if str(c) != 'nan' and len(str(c)) > 1]

unique_classifiers

In [ ]:
cate_keywords = {}
for c in unique_classifiers:
    class_df = cat_contracts.loc[cat_contracts['category'] == c]

    class_ls = class_df['description_en'].tolist()

    class_str = ' '.join(str(w) for w in class_ls).lower()
    class_str = clean_desc(class_str)
    class_str = [t for t in class_str if t.isalpha()]

    dist = FreqDist(class_str)

    word_freq = [ws[0] for ws in dist.most_common(15)]
    
    cate_keywords[c] = word_freq
    print(c)
    print(cate_keywords[c])

In [ ]:
# identify duplicates in keyword lists so they can be removed

# using sets to ensure no duplicates
distinct = set()
duplicate = set()

for k, v in cate_keywords.items():
  for i in set(v):
    if i in distinct:
      duplicate.add(i)
    else:
      distinct.add(i)

print(distinct)
print(duplicate)

In [ ]:
# TO DO
# adding vague words like "fees" and "misc" to words that should be removed to avoid confusion
# tbh maybe topic model or text classifier

In [ ]:
# remove duplicates from keyword lists
for k, v in cate_keywords.items():
    print(k)
    print(v)
    intersection = set(v) - duplicate
    print(intersection)
    print('------------------')
    cate_keywords[k] = list(intersection)

In [ ]:
cate_keywords

In [ ]:
count = 0
match_list = []
for row_index, cate in enumerate(cat_contracts['category']):
    # checking len of category name bc dynamic typing is great but also sucks
    if len(str(cate)) <= 3:
        tokenize_desc = clean_desc(cat_contracts.loc[row_index, 'description_en'])
        if tokenize_desc is not None:
            for k, v in cate_keywords.items():
                print(k)
                intersection = set(tokenize_desc).intersection(v)
                print(intersection)
                if len(intersection) > 0:
                    match_list.append(k)
            if len(match_list) > 1:
                print(tokenize_desc)
                print('MATCH: ' + str(match_list))
    print('--------------------')
    match_list = []
